Loading relevant packages

In [41]:
import requests
import json
from json import loads
import pandas as pd
from datetime import datetime
import time
import util

Using BART API to find relevant station information (Station name, Lattitude & Longitude for mapping)

In [ ]:
df = util.stationLoad(["San Francisco", "Oakland"])


                             name           city gtfs_latitude gtfs_longitude
0    12th St. Oakland City Center        Oakland     37.803768    -122.271450
1   Oakland International Airport        Oakland     37.713238    -122.212191
2                       MacArthur        Oakland     37.829065    -122.267040
3                    Lake Merritt        Oakland     37.797027    -122.265180
4                       Rockridge        Oakland     37.844702    -122.251371
5                        Coliseum        Oakland     37.753661    -122.196869
6                       Fruitvale        Oakland     37.774836    -122.224175
7                19th St. Oakland        Oakland     37.808350    -122.268602
8                    West Oakland        Oakland     37.804872    -122.295140
9                     Embarcadero  San Francisco     37.792874    -122.397020
10                    Balboa Park  San Francisco     37.721585    -122.447506
11                      Glen Park  San Francisco     37.733064  

In [44]:
df

,name,city,gtfs_latitude,gtfs_longitude
0,12th St. Oakland City Center,Oakland,37.803768,-122.271450
1,Oakland International Airport,Oakland,37.713238,-122.212191
2,MacArthur,Oakland,37.829065,-122.267040
3,Lake Merritt,Oakland,37.797027,-122.265180
4,Rockridge,Oakland,37.844702,-122.251371
5,Coliseum,Oakland,37.753661,-122.196869
6,Fruitvale,Oakland,37.774836,-122.224175
7,19th St. Oakland,Oakland,37.808350,-122.268602
8,West Oakland,Oakland,37.804872,-122.295140
9,Embarcadero,San Francisco,37.792874,-122.397020
